In [4]:
import cv2, os, tensorflow as tf, numpy as np
from tensorflow.python.keras.utils.data_utils import get_file

In [5]:
class_file="coco.names"
#modelURL="http://download.tensorflow.org/models/object_detection/tf2/20200711/efficientdet_d4_coco17_tpu-32.tar.gz"
modelURL="http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_320x320_coco17_tpu-8.tar.gz"

In [6]:
with open(class_file,'r') as file:
    class_list=file.read().splitlines()
class_list

['__Background__',
 'person',
 'bicycle',
 'car',
 'motorcycle',
 'airplane',
 'bus',
 'train',
 'truck',
 'boat',
 'traffic light',
 'fire hydrant',
 'street sign',
 'stop sign',
 'parking meter',
 'bench',
 'bird',
 'cat',
 'dog',
 'horse',
 'sheep',
 'cow',
 'elephant',
 'bear',
 'zebra',
 'giraffe',
 'hat',
 'backpack',
 'umbrella',
 'shoe',
 'eye glasses',
 'handbag',
 'tie',
 'suitcase',
 'frisbee',
 'skis',
 'snowboard',
 'sports ball',
 'kite',
 'baseball bat',
 'baseball glove',
 'skateboard',
 'surfboard',
 'tennis racket',
 'bottle',
 'plate',
 'wine glass',
 'cup',
 'fork',
 'knife',
 'spoon',
 'bowl',
 'banana',
 'apple',
 'sandwich',
 'orange',
 'broccoli',
 'carrot',
 'hot dog',
 'pizza',
 'donut',
 'cake',
 'chair',
 'couch',
 'potted plant',
 'bed',
 'mirror',
 'dining table',
 'window',
 'desk',
 'toilet',
 'door',
 'tv',
 'laptop',
 'mouse',
 'remote',
 'keyboard',
 'cell phone',
 'microwave',
 'oven',
 'toaster',
 'sink',
 'refrigerator',
 'blender',
 'book',
 'cloc

In [7]:
color_list=np.round(np.random.uniform(0,255,(len(class_list))))
color_list

array([107., 131., 155., 202., 216., 137.,  38.,  98.,  40., 116., 179.,
        37.,   6.,  34., 123.,  53., 200.,  41., 140.,  18., 220.,  25.,
       170., 250., 169., 149., 138., 247.,  34.,  46., 224.,  71., 138.,
       115., 250., 137., 131., 138., 178., 245., 127., 128., 219., 235.,
       242., 222., 189.,  28., 161.,  87.,  86., 168.,  46., 113., 212.,
       116.,  53.,  50., 118., 224., 223., 232., 190.,   6., 153., 182.,
       108.,  60., 163., 163.,  59., 160., 143.,  47.,  66.,  64.,  89.,
         5., 188., 238., 150.,  11., 185., 252., 199.,  92.,  91.,  55.,
       181., 228.,  14., 114.])

In [8]:
len(class_list)

92

In [9]:
len(color_list)

92

In [10]:
#download and exctract the model
model_file=os.path.basename(modelURL)
model_file

'ssd_mobilenet_v2_320x320_coco17_tpu-8.tar.gz'

In [11]:
model_name=model_file[0:model_file.index(".")]
model_name

'ssd_mobilenet_v2_320x320_coco17_tpu-8'

In [12]:
model_dir='./pretrained_models'
os.makedirs(model_dir,exist_ok=True)

In [13]:
get_file(fname=model_file,origin=modelURL,cache_dir=model_dir,cache_subdir='checkpoints',extract=True)

'./pretrained_models\\checkpoints\\ssd_mobilenet_v2_320x320_coco17_tpu-8.tar.gz'

In [14]:
tfod_model=tf.saved_model.load(os.path.join(model_dir,"checkpoints",model_name,"saved_model"))
print(str(tfod_model)," loaded ....")

<tensorflow.python.saved_model.load.Loader._recreate_base_user_object.<locals>._UserObject object at 0x000002108C0A1F60>  loaded ....


In [155]:
#read image data
image_path="test/2.jpg"
image=cv2.imread(image_path)

iH,iW,iC=image.shape
iH,iW,iC
image=cv2.resize(image,[int(iW/4),int(iH/4)])
cv2.imshow("test image",image)
cv2.waitKey(10000)
cv2.destroyAllWindows()

In [156]:
type(image)

numpy.ndarray

In [157]:
#preprocessing an image
image_1=cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
image_tensor=tf.convert_to_tensor(image_1,dtype=tf.uint8)
image_tensor=image_tensor[tf.newaxis,...]

In [158]:
image_tensor

<tf.Tensor: shape=(1, 876, 646, 3), dtype=uint8, numpy=
array([[[[50, 66, 30],
         [45, 64, 26],
         [45, 65, 26],
         ...,
         [50, 66, 37],
         [55, 70, 38],
         [56, 70, 35]],

        [[61, 74, 37],
         [56, 72, 34],
         [52, 70, 32],
         ...,
         [49, 64, 35],
         [49, 64, 37],
         [56, 69, 40]],

        [[64, 79, 40],
         [58, 75, 37],
         [52, 73, 35],
         ...,
         [52, 65, 36],
         [47, 62, 36],
         [54, 67, 40]],

        ...,

        [[73, 59, 49],
         [68, 59, 45],
         [68, 61, 47],
         ...,
         [59, 64, 47],
         [59, 59, 42],
         [75, 73, 56]],

        [[71, 66, 50],
         [65, 62, 44],
         [66, 61, 46],
         ...,
         [59, 63, 45],
         [64, 64, 46],
         [66, 68, 50]],

        [[59, 61, 43],
         [63, 62, 48],
         [63, 56, 44],
         ...,
         [57, 58, 43],
         [60, 62, 45],
         [53, 60, 43]]]], dtype

In [159]:
image_tensor.shape

TensorShape([1, 876, 646, 3])

In [160]:
#prection on image
detections=tfod_model(image_tensor)
detections

{'detection_multiclass_scores': <tf.Tensor: shape=(1, 100, 91), dtype=float32, numpy=
 array([[[0.0073181 , 0.85751873, 0.01555217, ..., 0.03936834,
          0.01429136, 0.01423877],
         [0.00330862, 0.03441056, 0.13739935, ..., 0.01116456,
          0.02357286, 0.01758344],
         [0.0030403 , 0.04068687, 0.00595981, ..., 0.00668035,
          0.00462165, 0.01478269],
         ...,
         [0.00307144, 0.03701049, 0.02680686, ..., 0.01151409,
          0.00964308, 0.01882408],
         [0.00348894, 0.03184248, 0.01260271, ..., 0.00734566,
          0.0241391 , 0.06651093],
         [0.00358579, 0.03603034, 0.00856517, ..., 0.00475167,
          0.01650365, 0.01565764]]], dtype=float32)>,
 'detection_anchor_indices': <tf.Tensor: shape=(1, 100), dtype=float32, numpy=
 array([[1857.,  502.,  555.,  193.,  897.,  193., 1362.,  897.,  141.,
          193.,  897.,  135.,  192.,  138.,  192.,  251.,  493.,  898.,
          196.,  555.,  196.,  189.,  192.,  897.,  444.,  898.,  445.

In [161]:
detections.keys()

dict_keys(['detection_multiclass_scores', 'detection_anchor_indices', 'raw_detection_boxes', 'detection_classes', 'detection_boxes', 'detection_scores', 'num_detections', 'raw_detection_scores'])

In [162]:
detections['detection_classes']

<tf.Tensor: shape=(1, 100), dtype=float32, numpy=
array([[ 1., 43., 37., 39., 34., 34., 43., 44., 34., 43., 42., 34., 77.,
        34., 34., 39., 39., 31., 28., 44., 77., 40., 39., 38., 43., 38.,
         2., 34., 37., 77., 44., 40., 41., 75., 34., 31., 33., 39., 84.,
        75., 85., 89., 31., 39.,  9., 27., 43., 37., 27., 37.,  1., 42.,
        37., 32., 34., 43., 77., 36., 77., 43., 43., 42., 39., 37., 43.,
        37., 41., 43., 43., 75., 43., 39.,  3., 62., 41., 40., 37., 43.,
        39., 39., 37., 53., 37., 35., 37., 77., 47., 42., 36., 87., 40.,
        33., 87., 43., 41., 11., 43., 37., 40., 37.]], dtype=float32)>

In [163]:
detections['num_detections']

<tf.Tensor: shape=(1,), dtype=float32, numpy=array([100.], dtype=float32)>

In [164]:
detections['detection_boxes']

<tf.Tensor: shape=(1, 100, 4), dtype=float32, numpy=
array([[[0.10152799, 0.22066903, 0.911697  , 0.8114953 ],
        [0.34645572, 0.6676787 , 0.51135635, 0.91765785],
        [0.4934278 , 0.71670365, 0.52613   , 0.7603743 ],
        [0.10275202, 0.29240674, 0.23228621, 0.5202436 ],
        [0.78279424, 0.68691933, 0.8421308 , 0.89954114],
        [0.10275202, 0.29240674, 0.23228621, 0.5202436 ],
        [0.3543751 , 0.5474214 , 0.5284181 , 0.90289587],
        [0.78279424, 0.68691933, 0.8421308 , 0.89954114],
        [0.11160795, 0.44444203, 0.151085  , 0.5190614 ],
        [0.10275202, 0.29240674, 0.23228621, 0.5202436 ],
        [0.78279424, 0.68691933, 0.8421308 , 0.89954114],
        [0.10011197, 0.3065942 , 0.19296983, 0.47066712],
        [0.11556753, 0.31232804, 0.22308266, 0.4575538 ],
        [0.09576003, 0.3784179 , 0.16781113, 0.52781653],
        [0.11556753, 0.31232804, 0.22308266, 0.4575538 ],
        [0.10523262, 0.2995662 , 0.3243579 , 0.4992544 ],
        [0.37292764

In [165]:
detections['detection_boxes'][0,0]

<tf.Tensor: shape=(4,), dtype=float32, numpy=array([0.10152799, 0.22066903, 0.911697  , 0.8114953 ], dtype=float32)>

In [166]:
iH,iW,iC=image.shape
iH,iW,iC

(876, 646, 3)

In [167]:
detections['detection_boxes'][0]

<tf.Tensor: shape=(100, 4), dtype=float32, numpy=
array([[0.10152799, 0.22066903, 0.911697  , 0.8114953 ],
       [0.34645572, 0.6676787 , 0.51135635, 0.91765785],
       [0.4934278 , 0.71670365, 0.52613   , 0.7603743 ],
       [0.10275202, 0.29240674, 0.23228621, 0.5202436 ],
       [0.78279424, 0.68691933, 0.8421308 , 0.89954114],
       [0.10275202, 0.29240674, 0.23228621, 0.5202436 ],
       [0.3543751 , 0.5474214 , 0.5284181 , 0.90289587],
       [0.78279424, 0.68691933, 0.8421308 , 0.89954114],
       [0.11160795, 0.44444203, 0.151085  , 0.5190614 ],
       [0.10275202, 0.29240674, 0.23228621, 0.5202436 ],
       [0.78279424, 0.68691933, 0.8421308 , 0.89954114],
       [0.10011197, 0.3065942 , 0.19296983, 0.47066712],
       [0.11556753, 0.31232804, 0.22308266, 0.4575538 ],
       [0.09576003, 0.3784179 , 0.16781113, 0.52781653],
       [0.11556753, 0.31232804, 0.22308266, 0.4575538 ],
       [0.10523262, 0.2995662 , 0.3243579 , 0.4992544 ],
       [0.37292764, 0.5564938 , 0.5273

In [168]:
bboxs=detections['detection_boxes'][0].numpy()
bboxs

array([[0.10152799, 0.22066903, 0.911697  , 0.8114953 ],
       [0.34645572, 0.6676787 , 0.51135635, 0.91765785],
       [0.4934278 , 0.71670365, 0.52613   , 0.7603743 ],
       [0.10275202, 0.29240674, 0.23228621, 0.5202436 ],
       [0.78279424, 0.68691933, 0.8421308 , 0.89954114],
       [0.10275202, 0.29240674, 0.23228621, 0.5202436 ],
       [0.3543751 , 0.5474214 , 0.5284181 , 0.90289587],
       [0.78279424, 0.68691933, 0.8421308 , 0.89954114],
       [0.11160795, 0.44444203, 0.151085  , 0.5190614 ],
       [0.10275202, 0.29240674, 0.23228621, 0.5202436 ],
       [0.78279424, 0.68691933, 0.8421308 , 0.89954114],
       [0.10011197, 0.3065942 , 0.19296983, 0.47066712],
       [0.11556753, 0.31232804, 0.22308266, 0.4575538 ],
       [0.09576003, 0.3784179 , 0.16781113, 0.52781653],
       [0.11556753, 0.31232804, 0.22308266, 0.4575538 ],
       [0.10523262, 0.2995662 , 0.3243579 , 0.4992544 ],
       [0.37292764, 0.5564938 , 0.52737206, 0.84557813],
       [0.7560337 , 0.69168085,

In [169]:
class_indexes=detections['detection_classes'][0].numpy().astype(np.int32)
class_indexes

array([ 1, 43, 37, 39, 34, 34, 43, 44, 34, 43, 42, 34, 77, 34, 34, 39, 39,
       31, 28, 44, 77, 40, 39, 38, 43, 38,  2, 34, 37, 77, 44, 40, 41, 75,
       34, 31, 33, 39, 84, 75, 85, 89, 31, 39,  9, 27, 43, 37, 27, 37,  1,
       42, 37, 32, 34, 43, 77, 36, 77, 43, 43, 42, 39, 37, 43, 37, 41, 43,
       43, 75, 43, 39,  3, 62, 41, 40, 37, 43, 39, 39, 37, 53, 37, 35, 37,
       77, 47, 42, 36, 87, 40, 33, 87, 43, 41, 11, 43, 37, 40, 37])

In [170]:
class_scores=detections['detection_scores'][0].numpy()
class_scores

array([0.85751873, 0.76324606, 0.72078145, 0.3033871 , 0.2789036 ,
       0.27321643, 0.25793955, 0.24551585, 0.23898602, 0.23512785,
       0.2282786 , 0.21786423, 0.21009047, 0.20098172, 0.1974149 ,
       0.19524853, 0.1827945 , 0.17775582, 0.17608672, 0.17201278,
       0.15719211, 0.15638106, 0.15344283, 0.15272251, 0.15181676,
       0.1505841 , 0.1504529 , 0.14947347, 0.14883618, 0.14855924,
       0.14483562, 0.14088017, 0.1373707 , 0.13651614, 0.13498384,
       0.1341921 , 0.13402411, 0.13310938, 0.13236752, 0.12888671,
       0.12673567, 0.12655038, 0.12173173, 0.12029645, 0.11997344,
       0.1187899 , 0.11720852, 0.11539269, 0.11490001, 0.1143454 ,
       0.11399388, 0.11264945, 0.11200511, 0.11173311, 0.11061011,
       0.10787164, 0.10668258, 0.10507289, 0.10469002, 0.10403606,
       0.10395951, 0.10383955, 0.10342381, 0.10313187, 0.10263397,
       0.10217424, 0.10073487, 0.0998751 , 0.09925383, 0.09913782,
       0.09878027, 0.0982248 , 0.09803658, 0.09787212, 0.09772

In [171]:
class_name=class_list[class_indexes[0]]
class_name

'person'

In [172]:
class_color=color_list[class_indexes[0]]
class_color

131.0

In [173]:
#get values of bounding box and de-normalise them
bboxs[0]

array([0.10152799, 0.22066903, 0.911697  , 0.8114953 ], dtype=float32)

In [174]:
y1,x1,y2,x2=bboxs[0]

In [175]:
y1,x1,y2,x2

(0.10152799, 0.22066903, 0.911697, 0.8114953)

In [176]:
int(0.051834047*iH)

45

In [177]:
iW,iH

(646, 876)

In [178]:
y1,x1,y2,x2=int(y1*iH),int(x1*iW),int(y2*iH),int(x2*iW)
y1,x1,y2,x2

(88, 142, 798, 524)

In [95]:
cv2.rectangle(image,(x1,y1),(x2,y2),(0, 191, 255),1)
cv2.putText(image,class_name,(20,20),cv2.FONT_HERSHEY_SIMPLEX,1,(51, 170,255),1)
cv2.imshow("Prediction",image)
cv2.waitKey(20000)
cv2.destroyAllWindows()

In [179]:
#bbox_idx=tf.image.non_max_supression(bboxs,class_scores,max_output_size=50,iou_treshold=0.5,score_threshold=0.5)

#if bbox is found i.e. object is detected
if len(bboxs)!=0:
    for i in range(0,len(bboxs)):
        bbox=tuple(bboxs[i].tolist())
        class_confidence=round(100*class_scores[i])
        class_index=class_indexes[i]
        
        if class_confidence>50.0:            
            print("class_confidence--",class_confidence)
            class_lbl_txt=class_list[class_index]
            class_color=color_list[class_index]

            display_text=str(class_lbl_txt)+" "+str(class_confidence)

            #get values of bbox
            #below values are normalised x,y cordinates
            ymin,xmin,ymax,xmax=bbox
            #converting them to actual x,y values
            xmin,xmax,ymin,ymax=(xmin*iW,xmax*iW,ymin*iH,ymax*iH)
            #converting flaot to int
            xmin,xmax,ymin,ymax=int(xmin),int(xmax),int(ymin),int(ymax)

            cv2.rectangle(image,(xmin,ymin),(xmax,ymax), color=(0, 191, 255),thickness=1)
            cv2.putText(image,display_text,(xmin,ymin-10),cv2.FONT_HERSHEY_PLAIN,1.5,(0, 191, 255),2)

    cv2.imshow("result",image)
    cv2.waitKey(20000)
    cv2.destroyAllWindows()
    cv2.imwrite("p5.jpg",image)

class_confidence-- 86
class_confidence-- 76
class_confidence-- 72
